In [1]:
# General imports
import numpy as np
import torch
import random

# DeepMoD functions
import matplotlib.pylab as plt
from deepymod import DeepMoD
from deepymod.model.func_approx import NN
from deepymod.model.library import Library2D_third
from deepymod.model.constraint import LeastSquares
from deepymod.model.sparse_estimators import Threshold,PDEFIND
from deepymod.training import train
from deepymod.training.sparsity_scheduler import TrainTestPeriodic
from scipy.io import loadmat

# Settings for reproducibility
np.random.seed(1)
torch.manual_seed(1)

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [ ]:
data = loadmat('../../Diffusion_2D_space81.mat')
data = np.real(data['Expression1']).reshape((81,81,81,4))[:,:,:,3]

width, width_2, steps = data.shape
x_arr = np.linspace(0,1,width)
y_arr = np.linspace(0,1,width_2)
t_arr = np.linspace(0,1,steps)
x_grid, y_grid, t_grid = np.meshgrid(x_arr, y_arr, t_arr, indexing='ij')

for idn in [3,5,7,9,11,14,21]:
    number_of_samples = idn
    tot_samples = number_of_samples*number_of_samples
    x_grid_new = np.empty([number_of_samples,number_of_samples,steps])
    y_grid_new = np.empty([number_of_samples,number_of_samples,steps])
    t_grid_new = np.empty([number_of_samples,number_of_samples,steps])
    data_new = np.empty([number_of_samples,number_of_samples,steps])
    for i in np.arange(steps):
        list_x = [random.sample(range(1,width),number_of_samples) for i in np.arange(0,number_of_samples)]
        list_y = [random.sample(range(1,width_2),number_of_samples) for i in np.arange(0,number_of_samples)]
        x_grid_new[:,:,i] = x_grid[list_x,list_y,i]
        y_grid_new[:,:,i] = y_grid[list_x,list_y,i]
        t_grid_new[:,:,i] = t_grid[list_x,list_y,i]
        data_new[:,:,i] = data[list_x,list_y,i]
    X = np.transpose((t_grid_new.flatten(), x_grid_new.flatten(), y_grid_new.flatten()))
    y = np.float32(data_new.reshape((data_new.size, 1)))

    # Add noise 
    noise_level = 0.0
    y_noisy = y + noise_level * np.std(y) * np.random.randn(y.size, 1)
    # Randomize data 

    idx = np.random.permutation(y.shape[0])
    X_train = torch.tensor(X[idx, :], dtype=torch.float32, requires_grad=True).to(device)
    y_train = torch.tensor(y_noisy[idx], dtype=torch.float32).to(device)

    # Configure DeepMoD

    network = NN(3, [40, 40, 40, 40], 1)
    library = Library2D_third(poly_order=0) 
    estimator = Threshold(0.05) 
    sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
    constraint = LeastSquares() 
    model = DeepMoD(network, library, estimator, constraint).to(device)
    optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=2e-3) 
    logdir='noise_0_random/x_' + str(idn)
    train(model, X_train, y_train, optimizer,sparsity_scheduler, log_dir=logdir, split=0.8, max_iterations=50000, delta=1e-5, patience=200) 


 49975  MSE: 2.81e-05  Reg: 5.92e-06  L1: 1.40e+00 Algorithm converged. Writing model to disk.
 49975  MSE: 8.75e-07  Reg: 1.27e-06  L1: 1.52e+00 Algorithm converged. Writing model to disk.
 40025  MSE: 1.80e-06  Reg: 2.74e-06  L1: 1.33e+00 Algorithm converged. Writing model to disk.
 49975  MSE: 3.08e-05  Reg: 1.40e-05  L1: 1.52e+00 Algorithm converged. Writing model to disk.
 49975  MSE: 7.15e-07  Reg: 2.34e-06  L1: 1.35e+00 Algorithm converged. Writing model to disk.
 21300  MSE: 1.49e-04  Reg: 3.24e-05  L1: 1.55e+00 Algorithm converged. Writing model to disk.
 39825  MSE: 1.34e-05  Reg: 8.49e-06  L1: 1.50e+00 